# Leveraging Video-On-Demand streaming data for early forecast of a movie's success using Gradient Boosting Machines and advanced feature engineering techniques.
## Introduction
The goal of this project is to create a decision support system to aid movie investments at the early stage of a movie's production. The system predicts the success of a movie based on a streaming rank scoring measure by leveraging historical data from various sources. Using social network analysis and advanced natural language processing (NLP) techniques, the system automatically extracts several groups of features, including the “who” (cast and crew), the “what” (the plot)), as well as “hybrid” features that match “who” with “what”. In order to support investment decisions on a movie, the model has to be provided information that is available at the very early stage of the movie’s production. Consequently, our prediction of movie success can only leverage data that is available when a movie is still being planned. Predictions that are made right before or after the official release may have more data to use and get more accurate results, but they are too late for investors to make any meaningful decision.

# Setup

## Python libraries

In [27]:
import pandas as pd
import numpy as np
from tqdm import tqdm
#from sqlalchemy import create_engine
#import requests
#import json
#import time
#import difflib
from random import choice
from itertools import combinations
from TextRank4Keyword import TextRank4Keyword # PageRank based keyword extraction
# import stopwords from nltk
import nltk
import networkx as nx # Graph analyses
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import lightgbm as lgb # Prediction model
#import edl
#from gspread_pandas import Spread
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#import os

nltk.download('stopwords')
from nltk.corpus import stopwords

data_path = './data/'
countries = ['Mexico', 'Brazil', 'United States'] # Can be expanded to other countries


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alejandroleda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Preprocessing

In [43]:

pred_set

,title,name,role,plot,age_certification,genre_1,genre_2,genre_3,comentarios_vivi,budget,ask,sales,market,status
0,WAY DOWN,Liam Cunningham,ACTOR,The Bank of Spain is like no other. An absolut...,PG-13,action,thriller,NaN,NaN,NaN,NaN,TF1,NaN,NaN
1,NaN,Astrid Bergès-Frisbey,ACTOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Freddie Highmore,ACTOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Jaume Balagueró,DIRECTOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Álvaro Augustín,PRODUCER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5562,NaN,Black Bear,PRODUCER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5563,NaN,JIGSAW PRODUCTIONS,PRODUCER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5564,NaN,CLOSER MEDIA,PRODUCER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5565,NaN,ANONYMOUS CONTENT.,PRODUCER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
pred_set = pd.read_csv(data_path + 'project_form - movie.csv')

# make all column names lowercase
pred_set.columns = map(str.lower, pred_set.columns)

mask = ~pred_set['title'].isna()
pred_set['title'] = pred_set['title'].ffill()

# Create a 'pred_set_talent' dataframe that contains only the 'title', 'name', and 'role' columns
pred_set_talent = pred_set[['title', 'name', 'role']]

pred_set_talent.dropna(inplace=True)
pred_set_talent

/var/folders/8y/0ldkfh015gjc7lvpvxd7pr380000gn/T/ipykernel_19259/1953403653.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_set_talent.dropna(inplace=True)


,title,name,role
0,WAY DOWN,Liam Cunningham,ACTOR
1,WAY DOWN,Astrid Bergès-Frisbey,ACTOR
2,WAY DOWN,Freddie Highmore,ACTOR
3,WAY DOWN,Jaume Balagueró,DIRECTOR
4,WAY DOWN,Álvaro Augustín,PRODUCER
...,...,...,...
5561,MUSK,ALEX GIBNEY,DIRECTOR
5562,MUSK,Black Bear,PRODUCER
5563,MUSK,JIGSAW PRODUCTIONS,PRODUCER
5564,MUSK,CLOSER MEDIA,PRODUCER


In [51]:
pred_set = pred_set[mask].drop(columns=['name', 'role'])

# Drop rows with missing values for 'plot'
pred_set = pred_set.dropna(subset=['plot'])

pred_set

,title,plot,age_certification,genre_1,genre_2,genre_3,comentarios_vivi,budget,ask,sales,market,status
0,WAY DOWN,The Bank of Spain is like no other. An absolut...,PG-13,action,thriller,NaN,NaN,NaN,NaN,TF1,NaN,NaN
7,THE GOOD BOSS,It’s a sharp and nuanced dark comedy about the...,PG-13,comedy,drama,NaN,"Es una comedia negra, por momentos bastante di...","3,000,000","250,000",MK2,NaN,NaN
9,LOIS WAIN,Louis Wain: Unconventional while iconic. Candi...,R,drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,RIO,"Set against the exotic backdrop of Brazil, thi...",R,thriller,action,NaN,NaN,NaN,NaN,STUDIOCANAL,EFM 2022,NaN
17,EMILY,Emily (Emma Mackey) wears a mask. The world te...,R,drama,NaN,NaN,Tipica historia tipo Pride and Prejudice. Se p...,Budget £8m,Asking para Latam: US$600k.,EMBANKMENT,Cannes 2022,Status Post-Production. Delivery Q1 2022
...,...,...,...,...,...,...,...,...,...,...,...,...
5540,CINNAMON,This darkly comedic heist thriller follows asp...,PG-13,thriller,comedy,NaN,NaN,NaN,Ask: 75K,VILLAGE ROADSHOW,CANNES 2023,NaN
5544,THE SALTED PATH,An honest and life-affirming true story of the...,R,drama,NaN,NaN,NaN,NaN,Ask: 450K,ROCKET SCIENCE,CANNES 2023,Pre prod. Shooting Date: 5th June 2023
5549,CONTROL,"Wallace Conway, a troubled doctor who increasi...",PG-13,thriller,drama,NaN,"Lei las primeras 40 paginas, es excelente, atr...",NaN,NaN,STUDIOCANAL,CANNES 2023,NaN
5553,CLIFFHANGER,Sylvester Stallone will reprise his character ...,PG-13,action,NaN,NaN,NaN,NaN,NaN,ROCKET SCIENCE,CANNES 2023,Shooting Date: September 2023


In [ ]:

    pred_kw = get_kw({'name': 'pred_set'}, pred_main[['title', 'plot']].set_index('title'), is_update=True)
    pred_main['is_nflx_original'] = False
    pred_main['is_pred'] = True


In [4]:
rank = pd.read_pickle(data_path + 'netflixmovie_rank.pkl')
titles = pd.read_pickle(data_path + 'titles.pkl')
talent = pd.read_pickle(data_path + 'talent.pkl')

In [5]:
main_df = pd.read_pickle(data_path + 'netflixmoviemain_df.pkl')
main_df

,country,jw_entity_id,rank,is_nflx_original,score,date,age_certification,object_type,original_release_year,original_title,...,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12
0,Argentina,tm1000599,NaN,None,1.0,2021-11-07,None,movie,2021.0,A Última Floresta,...,None,None,None,None,None,None,None,None,None,None
1,Argentina,tm1000619,NaN,None,1.0,2022-05-07,None,movie,2022.0,రాధే శ్యామ్,...,None,None,None,None,None,None,None,None,None,None
2,Argentina,tm1001097,NaN,None,1.0,2022-06-29,R,movie,2022.0,Beauty,...,romance,None,None,None,None,None,None,None,None,None
3,Argentina,tm1001912,NaN,None,1.0,2022-03-02,None,movie,2021.0,Trust,...,romance,None,None,None,None,None,None,None,None,None
4,Argentina,tm1003034,NaN,None,1.0,2021-08-23,None,movie,2021.0,The Witcher: Nightmare of the Wolf,...,scifi,animation,action,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25831,Venezuela,tm996762,NaN,None,1.0,2022-07-07,None,movie,2022.0,మేజర్,...,None,None,None,None,None,None,None,None,None,None
25832,Venezuela,tm998033,NaN,None,1.0,2021-11-18,None,movie,2021.0,டாக்டர்,...,comedy,crime,None,None,None,None,None,None,None,None
25833,Venezuela,tm998992,NaN,None,1.0,2022-09-07,PG,movie,2021.0,竜とそばかすの姫,...,fantasy,music,scifi,None,None,None,None,None,None,None
25834,Venezuela,tm999817,NaN,None,1.0,2021-12-01,None,movie,2021.0,白蛇 II：青蛇劫起,...,action,None,None,None,None,None,None,None,None,None


In [6]:
main_df.columns

Index(['country', 'jw_entity_id', 'rank', 'is_nflx_original', 'score', 'date',
       'age_certification', 'object_type', 'original_release_year',
       'original_title', 'production_countries', 'runtime',
       'short_description', 'title', 'localized_release_date', 'genres',
       'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6',
       'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12'],
      dtype='object')

In [37]:
talent

,role,character_name,person_id,name,title,jw_entity_id
0,ACTOR,Janaki,68294,Meena,Avvai Shanmugi,tm110160
1,ACTOR,Joseph,145348,Nagesh,Avvai Shanmugi,tm110160
2,ACTOR,Bhai,45436,Nassar,Avvai Shanmugi,tm110160
3,ACTOR,Rathna,432833,Heera Rajgopal,Avvai Shanmugi,tm110160
4,ACTOR,Kousi,471253,Rani,Avvai Shanmugi,tm110160
...,...,...,...,...,...,...
959963,EDITOR,None,15210,Larry Bock,Remember the Daze,tm73324
959964,EXECUTIVE_PRODUCER,None,618188,Kevin Loughery,Remember the Daze,tm73324
959965,ORIGINAL_MUSIC_COMPOSER,None,33249,Dustin O'Halloran,Remember the Daze,tm73324
959966,PRODUCER,None,17275,Matthew Rhodes,Remember the Daze,tm73324


In [38]:
talent = talent.merge(titles[['jw_entity_id', 'original_release_year', 'genre_1']], on='jw_entity_id', how='left')
talent

,role,character_name,person_id,name,title,jw_entity_id,original_release_year,genre_1
0,ACTOR,Janaki,68294,Meena,Avvai Shanmugi,tm110160,1996.0,drama
1,ACTOR,Joseph,145348,Nagesh,Avvai Shanmugi,tm110160,1996.0,drama
2,ACTOR,Bhai,45436,Nassar,Avvai Shanmugi,tm110160,1996.0,drama
3,ACTOR,Rathna,432833,Heera Rajgopal,Avvai Shanmugi,tm110160,1996.0,drama
4,ACTOR,Kousi,471253,Rani,Avvai Shanmugi,tm110160,1996.0,drama
...,...,...,...,...,...,...,...,...
959963,EDITOR,None,15210,Larry Bock,Remember the Daze,tm73324,2008.0,comedy
959964,EXECUTIVE_PRODUCER,None,618188,Kevin Loughery,Remember the Daze,tm73324,2008.0,comedy
959965,ORIGINAL_MUSIC_COMPOSER,None,33249,Dustin O'Halloran,Remember the Daze,tm73324,2008.0,comedy
959966,PRODUCER,None,17275,Matthew Rhodes,Remember the Daze,tm73324,2008.0,comedy


In [41]:
# Create a feature called 'tenure' to measure the number of years between the earliest movie and the latest movie of each talent
talent['tenure'] = talent.groupby('person_id')['original_release_year'].transform(lambda x: x.max() - x.min())


In [42]:
talent

,role,character_name,person_id,name,title,jw_entity_id,original_release_year,genre_1,tenure
0,ACTOR,Janaki,68294,Meena,Avvai Shanmugi,tm110160,1996.0,drama,35.0
1,ACTOR,Joseph,145348,Nagesh,Avvai Shanmugi,tm110160,1996.0,drama,46.0
2,ACTOR,Bhai,45436,Nassar,Avvai Shanmugi,tm110160,1996.0,drama,35.0
3,ACTOR,Rathna,432833,Heera Rajgopal,Avvai Shanmugi,tm110160,1996.0,drama,6.0
4,ACTOR,Kousi,471253,Rani,Avvai Shanmugi,tm110160,1996.0,drama,6.0
...,...,...,...,...,...,...,...,...,...
959963,EDITOR,None,15210,Larry Bock,Remember the Daze,tm73324,2008.0,comedy,30.0
959964,EXECUTIVE_PRODUCER,None,618188,Kevin Loughery,Remember the Daze,tm73324,2008.0,comedy,1.0
959965,ORIGINAL_MUSIC_COMPOSER,None,33249,Dustin O'Halloran,Remember the Daze,tm73324,2008.0,comedy,14.0
959966,PRODUCER,None,17275,Matthew Rhodes,Remember the Daze,tm73324,2008.0,comedy,16.0


In [49]:
# create a pandas series called 'plots' with the index as the movie's 'jw_entity_id' and the value as the movie's 'short_description'
plots = pd.Series(main_df['short_description'].values, index=main_df['jw_entity_id'])

# drop  duplicate index vales from the plots series
plots = plots[~plots.index.duplicated(keep='first')]

def score_keywords(plot_series):
    tr4w = TextRank4Keyword()
    keywords = pd.DataFrame()

    print('Extracting keywords from the plot of each of {} movies...'.format(len(plot_series)))

    # iterate through each movie's plot
    for i, row in tqdm(plot_series.iteritems()):
        try:
            tr4w.analyze(row, candidate_pos = ['NOUN'], window_size=4, lower=False)
        except TypeError:
            continue
        local_df = pd.DataFrame(tr4w.node_weight.items())
        local_df['jw_entity_id'] = i
        keywords = keywords.append(local_df, ignore_index=True)

    keywords.rename(columns={0:'keyword', 1:'node_weight'}, inplace=True)

    # Drop stopwords from the keywords dataframe
    keywords = keywords[~keywords['keyword'].isin(stopwords.words('english'))]

    # Drop punctuation from the keywords dataframe
    keywords = keywords[~keywords['keyword'].str.contains(r'[^\w\s]')]

    # Normalize node weights by dividing by the sum of all node weights for each movie
    keywords['node_weight_normalized'] = keywords.groupby('jw_entity_id')['node_weight'].transform(lambda x: x/x.sum())
    
    print('Total number of keywords extracted: {}'.format(len(keywords)))

    keywords_score = keywords.merge(main_df[['country', 'jw_entity_id', 'score']], on='jw_entity_id', how='inner')

    # Weigh the node weights by the movie's score
    keywords_score['node_weight_scored'] = keywords_score['node_weight_normalized'] * keywords_score['score']

    # Create a dataframe called 'keywords_scored_by_keyword_and_country' with the 'node_weight_scored' column summed by 'keyword' and 'country'
    keywords_scored_by_keyword_and_country = keywords_score.groupby(['keyword', 'country'])['node_weight_scored'].sum().reset_index()

    # Merge the 'keywords_scored' dataframe with the 'keywords_scored_by_keyword_and_country' dataframe on 'keyword' and 'country'
    keywords_scored = keywords_score.merge(keywords_scored_by_keyword_and_country, on=['keyword', 'country'], suffixes=('', '_by_keyword_and_country'))

    return keywords_scored

def score_plots(plot_series, keywords_scored):
    
    # Create a dataframe called 'plot_score' with the 'node_weight_scored_by_keyword_and_country' column summed by 'jw_entity_id' and 'country'
    plots_scored = keywords_scored.groupby(['jw_entity_id', 'country'])['node_weight_scored_by_keyword_and_country'].sum().reset_index()
    
    return plots_scored

scored_keywords = score_keywords(plots)
scored_plots = score_plots(plots, scored_keywords)
scored_plots


0it [00:00, ?it/s]

Extracting keywords from the plot of each of 4130 movies...


4130it [00:52, 79.19it/s]


Total number of keywords extracted: 41148


,jw_entity_id,country,node_weight_scored_by_keyword_and_country
0,tm10,Hungary,22.590855
1,tm10,India,167.322700
2,tm10,South Africa,126.902840
3,tm1000037,France,70.888013
4,tm1000037,Greece,90.606945
...,...,...,...
25786,tm999927,South Africa,17.881518
25787,tm999927,Thailand,109.000463
25788,tm999927,United Kingdom,25.946371
25789,tm999927,Venezuela,43.813079
